# Multiprocessing
***

Permite realizar processos em paralelo criando efetivamente um processo diferente de threads

**Vantagens**:

* Funciona em várias plataformas distintas diferente do **os.fork()**


* Perde em velocidade para threads porém efetivamente executa os programas em núcleos distintos da CPU

**Desvantagens**:

* Mudanças em um processo não afetam o outro


* Algumas estruturas (como o lambda) não podem ser rodadas em paralelo

Ela apresenta estruturas semelhantes aos modulos **threading** e **queue**

No Unix, usa um fork para criar um processo filho e invoca o método run de Process para roda-lo.

No windows um novo interpretador é criado usando o processo de criação de ferramentas do Windows passando um objeto codificado pelo **pickle** para um novo processo por meio de um pipe e começando  o **python -c** para rodar o programa que roda com uma função especial construida em python que lê e decodifica o objeto e invoca o método run().

Modulo popular para execução paralela: **Scipy**

Alguns comandos para mexer com processos:

```sh
# Mostra uma lista com todos os processos que estão sendo executados na maquina
ps -e
# Mostra uma lista mais detalhada desses processos
ps -ef
# Lista todos os processos como uma arvore hierarquica
pstree
# Filtra os processos pelo nome
ps -C <nome-do-processo>
# Deleta um processo
kill <id-do-processo>
kill -9 <id-do-processo>
```

In [1]:
import math
from time import time
import multiprocessing


def computer(start: int, end: int) -> None:
    """
    Realiza o pode computacional.
    """

    print(f"Iniciando o cálculo {multiprocessing.current_process().name}\n")

    i = start
    factor = 1000 * 1000
    while i < end:
        i += 1
        math.sqrt((i - factor) * (i - factor))
        
    print(f"Finalizando o cálculo {multiprocessing.current_process().name}\n")


In [2]:
start_time = time()
computer(1, 50_000_000)
end_time = time() - start_time
print(f"Requisição executada em {round(end_time, 2)} segundos")

Iniciando o cálculo MainProcess

Finalizando o cálculo MainProcess

Requisição executada em 22.55 segundos


***
### Processo Simples
***

Testando o uso de processos simples em cpu bound

In [3]:
from concurrent.futures.process import ProcessPoolExecutor

start_time = time()
with ProcessPoolExecutor() as executor:
    executor.submit(computer, start=1, end=50_000_000)

end_time = time() - start_time
print(f"Requisição executada em {round(end_time, 2)} segundos")

Iniciando o cálculo ForkProcess-1

Finalizando o cálculo ForkProcess-1

Requisição executada em 20.64 segundos


***
### Multiplos Processos
***

In [4]:
import multiprocessing
from concurrent.futures.process import ProcessPoolExecutor

qtd_cores = multiprocessing.cpu_count()
main_proccess = multiprocessing.current_process().name


def process_name_into_pool() -> str:
    """
    Mostra o processo que está sendo executado dentro da pool
    """

    name = multiprocessing.current_process().name
    print(f"Iniciando o processo: {name}")
    return name



start_time = time()
with ProcessPoolExecutor(max_workers=qtd_cores) as executor:
    for n in range(1, qtd_cores + 1):
        initial = 50_000_000 * (n - 1) / qtd_cores
        end = 50_000_000 * n / qtd_cores
        print(f"Core {n} processando de {initial} até {end} usando sub-processos")
        executor.submit(computer, start=initial, end=end)

end_time = time() - start_time
print(f"Requisição executada em {round(end_time, 2)} segundos")

Core 1 processando de 0.0 até 6250000.0 usando sub-processos
Iniciando o cálculo ForkProcess-9

Iniciando o cálculo ForkProcess-10
Iniciando o cálculo ForkProcess-11
Iniciando o cálculo ForkProcess-13
Iniciando o cálculo ForkProcess-12
Iniciando o cálculo ForkProcess-14


Iniciando o cálculo ForkProcess-15
Iniciando o cálculo ForkProcess-16





Core 2 processando de 6250000.0 até 12500000.0 usando sub-processos
Core 3 processando de 12500000.0 até 18750000.0 usando sub-processos
Core 4 processando de 18750000.0 até 25000000.0 usando sub-processos
Core 5 processando de 25000000.0 até 31250000.0 usando sub-processos
Core 6 processando de 31250000.0 até 37500000.0 usando sub-processos
Core 7 processando de 37500000.0 até 43750000.0 usando sub-processos
Core 8 processando de 43750000.0 até 50000000.0 usando sub-processos
Finalizando o cálculo ForkProcess-11

Finalizando o cálculo ForkProcess-13

Finalizando o cálculo ForkProcess-12

Finalizando o cálculo ForkProcess-9

Finalizando o cálcu

***
### Shared Memory
***

In [5]:
from multiprocessing import Process, current_process, Manager, Value, RLock as ProcessLock
from multiprocessing.synchronize import RLock
from time import time
import ctypes
import random

In [6]:
def modify_data(counter: ctypes.c_int, results: list[bool], objs: dict, lock: RLock) -> None:
    """
    Incrementa o contador, adicionar randomicamente um valor booleano na lista e eleva ao quadrado o contador.
    """

    with lock:
        results.append(random.choice([True, False]))
        counter.value = counter.value + 1
        objs[f"{counter.value}^2"] = counter.value ** 2
        process_name = current_process().name
        print(f"No processo [{process_name}] obtivemos: {counter.value}) {results[:]} e {objs}")

In [7]:
start_time = time()
print(f"Iniciando o processo {current_process().name}")
manager = Manager()
lock = ProcessLock()

counter = Value(ctypes.c_int, 0)
results: list[bool] = manager.list()
objs: dict = manager.dict()

p1 = Process(target=modify_data, args=(counter, results, objs, lock))
p2 = Process(target=modify_data, args=(counter, results, objs, lock))
p3 = Process(target=modify_data, args=(counter, results, objs, lock))
p4 = Process(target=modify_data, args=(counter, results, objs, lock))
p5 = Process(target=modify_data, args=(counter, results, objs, lock))

p1.start()
p2.start()
p3.start()
p4.start()
p5.start()

p1.join()
p2.join()
p3.join()
p4.join()
p5.join()

end_time = time() - start_time
print(f"Requisição executada em {round(end_time, 2)} segundos")

Iniciando o processo MainProcess
No processo [Process-18] obtivemos: 1) [True] e {'1^2': 1}
No processo [Process-19] obtivemos: 2) [True, False] e {'1^2': 1, '2^2': 4}
No processo [Process-20] obtivemos: 3) [True, False, False] e {'1^2': 1, '2^2': 4, '3^2': 9}
No processo [Process-21] obtivemos: 4) [True, False, False, True] e {'1^2': 1, '2^2': 4, '3^2': 9, '4^2': 16}
No processo [Process-22] obtivemos: 5) [True, False, False, True, False] e {'1^2': 1, '2^2': 4, '3^2': 9, '4^2': 16, '5^2': 25}
Requisição executada em 0.21 segundos


***
### Utilizando Pipes para comunicação unilateral entre processos
***

In [8]:
from multiprocessing import Pipe, Process, current_process
from multiprocessing.connection import Connection
from time import time

In [9]:
def sender(connection: Connection) -> None:
    """
    Envia msg
    """

    print(f"Enviando a mensagem pelo processo {current_process().name}.")
    connection.send('Hello')

In [10]:
def receiver(connection: Connection) -> None:
    """
    Recebe msg
    """

    msg = connection.recv()
    print(f"Mensagem recebida no processo {current_process().name}: {msg} World")

In [11]:
start_time = time()
print(f"Iniciando o processo {current_process().name}")
# Queremos uma conexão enviar e a outra só receber por isso duplex=False
# Se quisermos ambas enviar e receber usamos o duplex=True
connection_receiver, connection_sender = Pipe(duplex=False)

proccess_sender = Process(target=sender, args=(connection_sender,))
proccess_receiver = Process(target=receiver, args=(connection_receiver,))

proccess_sender.start()
proccess_receiver.start()

proccess_sender.join()
proccess_receiver.join()

end_time = time() - start_time
print(f"Requisição executada em {round(end_time, 2)} segundos")

Iniciando o processo MainProcess
Enviando a mensagem pelo processo Process-23.
Mensagem recebida no processo Process-24: Hello World
Requisição executada em 0.04 segundos


***
### Utilizando Queues para comunicação unilateral entre processos
***

In [12]:
from time import time
from multiprocessing import Process, Queue, current_process

In [13]:
def sender(queue: Queue) -> None:
    """
    Envia msg
    """

    print(f"Enviando a mensagem pelo processo {current_process().name}.")
    queue.put('Hello')

In [14]:
def receiver(queue: Queue) -> None:
    """
    Recebe msg
    """

    msg = queue.get()
    print(f"Recebendo mensagem no processo {current_process().name}: {msg} World")

In [15]:
start_time = time()
print(f"Iniciando o processo: {current_process().name}")
queue = Queue()

proccess_sender = Process(target=sender, args=(queue,))
proccess_receiver = Process(target=receiver, args=(queue,))

proccess_sender.start()
proccess_receiver.start()

proccess_sender.join()
proccess_receiver.join()

end_time = time() - start_time
print(f"Requisição executada em {round(end_time, 2)} segundos")

Iniciando o processo: MainProcess
Enviando a mensagem pelo processo Process-25.
Recebendo mensagem no processo Process-26: Hello World
Requisição executada em 0.04 segundos


***
### Comunicação bidirecional com pipe e eventos
***

In [16]:
from multiprocessing import Process, Pipe, current_process, Event as MultiprocessingEvent
from multiprocessing.synchronize import Event
from multiprocessing.connection import Connection
import time

In [17]:
def ping(pipe: Connection, stop_event: Event) -> None:
    """
    Envia e recebe mensagem
    """

    process_name = current_process().name

    while not stop_event.is_set():
        message: str = pipe.recv()
        if message.lower() == 'sair':
            pipe.send('sair')
            break

        pipe.send(message)
        print(f"Processo {process_name} enviou:", message)

In [18]:
def pong(pipe: Connection, stop_event: Event) -> None:
    """
    Envia e recebe mensagem.
    """

    process_name = current_process().name

    while not stop_event.is_set():
        message: str = pipe.recv()
        if message.lower() == 'sair':
            pipe.send('sair')
            break

        pipe.send(message)
        print(f"Processo {process_name} enviou:", message)

In [19]:
parent_ping, child_ping = Pipe()
parent_pong, child_pong = Pipe()
stop_event = MultiprocessingEvent()

process_ping = Process(name="ping", target=ping, args=(child_ping, stop_event))
process_pong = Process(name="pong", target=pong, args=(child_pong, stop_event))

process_ping.start()
process_pong.start()

while not stop_event.is_set():
    time.sleep(2)
    ping_input = input(f"Digite uma mensagem para enviar ao {process_ping.name} (ou 'sair' para terminar): ")
    if ping_input.lower() == 'sair':
        parent_ping.send('sair')
        parent_pong.send('sair')
        stop_event.set()
    else:
        parent_ping.send(ping_input)

    time.sleep(2)
    pong_input = input(f"Digite uma mensagem para enviar ao {process_pong.name} (ou 'sair' para terminar): ")
    if pong_input.lower() == 'sair':
        parent_ping.send('sair')
        parent_pong.send('sair')
        stop_event.set()
    else:
        parent_pong.send(pong_input)

process_ping.join()
process_pong.join()

print("Ping Pong concluido.")

Digite uma mensagem para enviar ao ping (ou 'sair' para terminar):  Ola pong


Processo ping enviou: Ola pong


Digite uma mensagem para enviar ao pong (ou 'sair' para terminar):  Ola ping


Processo pong enviou: Ola ping


Digite uma mensagem para enviar ao ping (ou 'sair' para terminar):  Como está seu dia pong?


...

Processo ping enviou: está ótimo, também


Digite uma mensagem para enviar ao pong (ou 'sair' para terminar):  que bom


Processo pong enviou: que bom


Digite uma mensagem para enviar ao ping (ou 'sair' para terminar):  sair
Digite uma mensagem para enviar ao pong (ou 'sair' para terminar):  sair


Ping Pong concluido.
